In [5]:
from langchain_core.retrievers import BaseRetriever
import hashlib
from langchain_core.documents import Document
from sentence_transformers import CrossEncoder
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

import re
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# API_KEY="lm-studio"
# API_BASE="http://localhost:1234/v1"
# MODEL="openai/gpt-oss-20b"
# #MODEL="unsloth/Devstral-Small-2-24B-Instruct-2512-GGUF"

load_dotenv('my.env')
MODEL = os.getenv("MODEL")
API_KEY = os.getenv("API_KEY")
API_BASE = os.getenv("API_BASE")

llm = ChatOpenAI(
    model=MODEL,
    openai_api_key=API_KEY,
    openai_api_base=API_BASE,
    temperature=0.05
)

class SimpleReranker:
    def __init__(self, model_name):
        self.model = CrossEncoder(model_name)

    def rerank(self, query, documents, top_n=3):
        if not documents:
            return []

        # Подготавливаем пары (query, document)
        pairs = [[query, doc.page_content] for doc in documents]

        # Получаем скоры релевантности
        scores = self.model.predict(pairs)

        # Сортируем документы по скору
        doc_score_pairs = list(zip(documents, scores))
        doc_score_pairs.sort(key=lambda x: x[1], reverse=True)

        # Отбираем топ-N документов (score записываем в метаданные)
        result = []
        for doc, score in doc_score_pairs[:top_n]:
            doc.metadata = doc.metadata or {}
            doc.metadata["rerank_score"] = float(score)
            result.append(doc)

        return result


class HybridRerankerRetriever(BaseRetriever):
    first_retriever: BaseRetriever
    second_retriever: BaseRetriever
    reranker: SimpleReranker
    k: int = 3

    def _dedup_docs(self, docs):
        '''объединение результатов с дедупликацией'''
        seen = set()
        result = []
        for d in docs:
            content = d.page_content
            uid = hashlib.md5(content.encode("utf-8")).hexdigest()
            if uid not in seen:
                seen.add(uid)
                result.append(d)
        return result

    def _get_relevant_documents(self, query):
        first_docs = self.first_retriever.invoke(query)
        second_docs = self.second_retriever.invoke(query)
        merged = self._dedup_docs(first_docs + second_docs)

        return self.reranker.rerank(query, merged, self.k)

def evaluate_qa_with_llm(query: str, prediction: str, reference: str) -> dict:
    prompt = f"""Ты эксперт по оценке качества ответов.

    Вопрос: {query}
    Ожидаемый ответ: {reference}
    Полученный ответ: {prediction}

    Оцени, насколько полученный ответ соответствует ожидаемому по смыслу.
    Ответь ТОЛЬКО одним словом: CORRECT или INCORRECT"""

    response = llm.invoke(prompt)
    verdict = response.content.strip().upper()
    score = 1.0 if verdict == "CORRECT" else 0.0

    return score, verdict


# 1. Готовые документы
docs = [
    Document(page_content="Джим Хокинс — главный герой и рассказчик, сын владельцев трактира «Адмирал Бенбоу», смелый и находчивый юноша", metadata={'source': '1'}),
    Document(page_content="Билли Бонс — старый пират, бывший штурман капитана Флинта, постоялец в трактире Хокинсов", metadata={'source': '2'}),
    Document(page_content="Джон Сильвер (Долговязый Джон) — одноногий кок, бывший квартирмейстер капитана Флинта, предводитель пиратов", metadata={'source': '3'}),
    Document(page_content="Доктор Ливси — благородный и мужественный врач, друг Джима", metadata={'source': '4'}),
    Document(page_content="Сквайр Трелони — богатый землевладелец, организатор экспедиции", metadata={'source': '5'}),
    Document(page_content="Капитан Смоллетт — честный и опытный капитан шхуны «Испаньола»", metadata={'source': '6'}),
    Document(page_content="Бен Ганн — бывший пират, брошенный товарищами на острове три года назад", metadata={'source': '7'}),
    Document(page_content="Чёрный Пёс — пират, разыскивающий Билли Бонса", metadata={'source': '8'}),
    Document(page_content="Слепой Пью — слепой пират, приносящий Билли Бонсу чёрную метку", metadata={'source': '9'}),
    Document(page_content="""В трактире «Адмирал Бенбоу» на побережье Англии поселяется странный постоялец — грузный старый моряк с сабельным шрамом на щеке, который представляется как капитан Билли Бонс. Он груб и пьёт ром, при этом явно кого-то боится. Билли Бонс просит Джима Хокинса, сына хозяев трактира, следить, не появится ли поблизости одноногий моряк.""", metadata={'source': '10'}),
    Document(page_content="""В трактир приходит незнакомец с бледным лицом по прозвищу Чёрный Пёс. Между ним и Билли Бонсом происходит ссора и драка. Чёрный Пёс, раненный в плечо, убегает. От волнения у Билли Бонса случается апоплексический удар.""", metadata={'source': '11'}),
    Document(page_content="""Доктор Ливси лечит капитана и предупреждает его о необходимости бросить пить. Билли Бонс рассказывает Джиму, что был штурманом у знаменитого пирата капитана Флинта и что его бывшие сообщники охотятся за содержимым его сундука. Он боится получить чёрную метку — пиратское предупреждение.""", metadata={'source': '12'}),
    Document(page_content="""Отец Джима умирает. В день похорон к трактиру приходит отвратительный слепец по имени Пью и передаёт Билли Бонсу чёрную метку. Старый пират пытается бежать, но умирает от сердечного приступа. Джим с матерью понимают, что пираты скоро придут за сундуком. Они вскрывают его, берут причитающиеся деньги за постой и таинственный пакет.""", metadata={'source': '13'}),
    Document(page_content="""Джим с матерью прячутся. Появляются пираты во главе со слепым Пью и обыскивают трактир, но не находят того, что ищут. Появляются таможенные стражники, пираты разбегаются, а слепой Пью погибает под копытами лошади.""", metadata={'source': '14'}),
    Document(page_content="""Джим отдаёт пакет доктору Ливси и сквайру Трелони. В нём оказывается карта острова, где спрятаны сокровища капитана Флинта. Джентльмены решают отправиться за кладом, взяв Джима юнгой.""", metadata={'source': '15'}),
    Document(page_content="""Сквайр Трелони уезжает в Бристоль, чтобы купить корабль и нанять команду. Несмотря на обещание хранить тайну, он рассказывает всему городу о предстоящей экспедиции за сокровищами.""", metadata={'source': '16'}),
    Document(page_content="""Джим прибывает в Бристоль. В таверне «Подзорная труба», принадлежащей одноногому Джону Сильверу, Джим замечает Чёрного Пса, который при виде мальчика убегает. Сильвер производит на Джима благоприятное впечатление.""", metadata={'source': '17'}),
    Document(page_content="""Капитан Смоллетт недоволен командой и тем, что все знают о цели плавания. Он не доверяет морякам, нанятым по рекомендации Джона Сильвера.""", metadata={'source': '18'}),
    Document(page_content="""Шхуна «Испаньола» отправляется в плавание. Джон Сильвер пользуется всеобщим уважением на корабле.""", metadata={'source': '19'}),
    Document(page_content="""Когда корабль приближается к острову, Джим случайно прячется в яблочной бочке и подслушивает разговор Сильвера с матросами. Он узнаёт, что большинство команды — пираты, а их главарь — одноногий кок, бывший квартирмейстер капитана Флинта. Пираты планируют захватить сокровища и убить всех честных людей на судне.""", metadata={'source': '20'}),
    Document(page_content="""Джим рассказывает капитану, доктору и сквайру об услышанном. Они понимают опасность ситуации и разрабатывают план действий.""", metadata={'source': '21'}),
    Document(page_content="""Корабль достигает острова. Дисциплина падает, назревает бунт. Капитан разрешает пиратам сойти на берег. Джим импульсивно прыгает в одну из шлюпок.""", metadata={'source': '22'}),
    Document(page_content="""Джим убегает от пиратов в лес. Он становится свидетелем того, как Джон Сильвер убивает честного матроса, отказавшегося присоединиться к мятежникам.""", metadata={'source': '23'}),
    Document(page_content="""Блуждая по острову, Джим встречает странного человека — это Бен Ганн, бывший пират, оставленный на острове три года назад. Бен Ганн говорит, что готов помогать джентльменам, и сообщает Джиму, что у него есть лодка.""", metadata={'source': '24'}),
    Document(page_content="""Доктор Ливси продолжает повествование. Честные люди с корабля — капитан, доктор, сквайр, три слуги (Хантер, Джойс и Редрут) и Абрахам Грей, помощник плотника, который отказался присоединиться к пиратам — на ялике перевозят на берег оружие, боеприпасы и провизию и укрываются в заброшенном форте за частоколом.""", metadata={'source': '25'}),
    Document(page_content="""Во время последнего рейса за припасами они подвергаются обстрелу пиратов, один из слуг сквайра (Том Редрут) погибает. Над фортом поднимается британский флаг.""", metadata={'source': '26'}),
    Document(page_content="""Увидев британский флаг, Джим понимает, где находятся друзья, и присоединяется к ним в форте. Он рассказывает о встрече с Беном Ганном.""", metadata={'source': '27'}),
    Document(page_content="""Осаждённые готовятся к обороне. Джим Хокинс снова ведёт рассказ. К форту приходит Джон Сильвер с предложением о переговорах.""", metadata={'source': '28'}),
    Document(page_content="""Сильвер требует карту сокровищ в обмен на жизнь. Капитан Смоллетт отказывается. Сильвер предупреждает о нападении.""", metadata={'source': '29'}),
    Document(page_content="""Пираты атакуют форт. Происходит жестокий бой. Защитники отбивают атаку, но несут потери. Среди защитников погибают и получают ранения несколько человек, капитан Смоллетт ранен.""", metadata={'source': '30'}),
    Document(page_content="""Доктор отправляется на встречу с Беном Ганном. Джим без разрешения покидает форт, находит лодку Бена Ганна и решает отрезать «Испаньолу» от якоря.""", metadata={'source': '31'}),
    Document(page_content="""Джим на хрупкой лодке плывёт к кораблю ночью и перерезает якорный канат. Корабль начинает дрейфовать.""", metadata={'source': '32'}),
    Document(page_content="""Утром Джим видит, что корабль дрейфует. Он пытается догнать его на своей лодке.""", metadata={'source': '33'}),
    Document(page_content="""Джим забирается на борт «Испаньолы». Он обнаруживает, что два пирата, охранявшие корабль, устроили пьяную драку: один мёртв, другой ранен. Джим берёт корабль под свой контроль и сбрасывает пиратский флаг.""", metadata={'source': '34'}),
    Document(page_content="""Раненый пират Израэль Хендс помогает Джиму управлять кораблём, но потом пытается убить мальчика. Джим застреливает его в самообороне. Он отводит корабль в укромную бухту.""", metadata={'source': '35'}),
    Document(page_content="""Джим возвращается в форт и обнаруживает, что он занят пиратами. Его берут в плен. Пираты хотят его убить, но Джон Сильвер неожиданно заступается за мальчика.""", metadata={'source': '36'}),
    Document(page_content="""Джим узнаёт, что доктор Ливси отдал пиратам форт и карту в обмен на неизвестные условия. Пираты недовольны Сильвером и готовы его сместить.""", metadata={'source': '37'}),
    Document(page_content="""Пираты вручают Сильверу чёрную метку, требуя его отставки. Сильвер показывает им карту сокровищ, что временно успокаивает бунтовщиков.""", metadata={'source': '38'}),
    Document(page_content="""Доктор Ливси приходит лечить раненых пиратов. Он разговаривает с Джимом и предлагает ему бежать, но Джим отказывается нарушить данное Сильверу слово. Доктор предупреждает Сильвера об опасности.""", metadata={'source': '39'}),
    Document(page_content="""Пираты отправляются искать клад по карте. По дороге они находят скелет, расположенный как указатель.""", metadata={'source': '40'}),
    Document(page_content="""Пираты слышат таинственный голос, который они принимают за голос мёртвого капитана Флинта. Сильвер убеждает их, что это просто чей-то розыгрыш.""", metadata={'source': '41'}),
    Document(page_content="""Пираты обнаруживают, что клад уже выкопан — яма пуста. Разъярённые пираты нападают на Сильвера и Джима, но раздаются выстрелы. Доктор Ливси, Бен Ганн и Абрахам Грей спасают их. Джордж Мерри был ранен выстрелом, а Сильвер его добил. Остальные пираты разбегаются.""", metadata={'source': '42'}),
    Document(page_content="""Выясняется, что Бен Ганн давно нашёл сокровища и перенёс их в свою пещеру. Там устроен настоящий склад золота. Сокровища грузят на корабль. По дороге в Англию, во время остановки в одном из портов Америки, Джон Сильвер сбегает, прихватив часть золота. Остальные благополучно возвращаются домой, где каждый получает свою долю клада. Бен Ганн промотал свою долю (тысячу фунтов) за девятнадцать дней и стал привратником у сквайра Трелони. Джим говорит, что никакие сокровища не заставят его вернуться на проклятый остров, который до сих пор снится ему в кошмарах.""", metadata={'source': '43'}),
    Document(page_content="""«Остров сокровищ» — это не просто приключенческий роман о поиске пиратского клада. Произведение Стивенсона исследует темы взросления, мужества и моральных выборов. Главный герой Джим Хокинс проходит путь от наивного мальчика до зрелого юноши, научившегося различать добро и зло.""", metadata={'source': '44'}),
    Document(page_content="""Центральный конфликт романа — противостояние между честностью и алчностью, законом и беззаконием. Джентльмены (доктор Ливси, капитан Смоллетт, сквайр Трелони) представляют цивилизованное общество с его правилами и моралью, тогда как пираты олицетворяют анархию и своеволие.""", metadata={'source': '45'}),
    Document(page_content="""Особенно интересен образ Джона Сильвера — сложного и противоречивого персонажа, который сочетает в себе обаяние, ум и жестокость. Он становится для Джима одновременно и врагом, и своеобразным учителем жизни. Сильвер демонстрирует, что в реальном мире грань между добром и злом не всегда очевидна.""", metadata={'source': '46'}),
    Document(page_content="""Роман также показывает, что жадность разрушительна: пираты готовы убивать друг друга из-за золота, а сокровища, которые должны были принести счастье, приносят лишь смерть и предательство. В конце Джим признаётся, что воспоминания об острове преследуют его в кошмарах, подчёркивая, что цена золота оказалась слишком высока.""", metadata={'source': '47'}),
    Document(page_content="""«Остров сокровищ» учит читателя ценить верность, честь и дружбу превыше материальных богатств, а также показывает, как важно сохранять человечность даже в самых опасных обстоятельствах. """, metadata={'source': '48'}),

    ]

# 2. ФОРМИРОВАНИЕ ground_truth_docs
ground_truth_docs = {
    "Вспомните имя главного героя книги 'Остров сокровищ'": ['1', '44'],
    "Как назывался трактир родителей Джима": ['1', '10'],
    "Кого Билли Бонс больше всего опасался?": ['8'],
    "Вспомните титул мистера Трелони Хоупа": ['5'],
    "Как назывался корабль, на котором герои книги отправились на остров?": ['6', '19'],
    "Какое название носил далекий остров?": ['44'],
    "Назовите имя пирата, чьи сокровища были спрятаны на острове.": ['15'],
    "Кто увел шхуну у пиратов и спрятал ее в бухте?": ['34', '35'],
    "Кто выкопал сокровища и спрятал их у себя?": ['43'],
    "Кого автор представлял как сторонников беззакония и смуты?": ['45', '46'],
}

predicted_answers = {
    "Вспомните имя главного героя книги 'Остров сокровищ'": 'Джим Хокинс',
    "Как назывался трактир родителей Джима": 'Адмирал Бенбоу',
    "Кого Билли Бонс больше всего опасался?": 'Чёрный Пёс',
    "Вспомните титул мистера Трелони Хоупа": 'Сквайр',
    "Как назывался корабль, на котором герои книги отправились на остров?": 'Испаньола',
    "Какое название носил далекий остров?": 'Остров сокровищ',
    "Назовите имя пирата, чьи сокровища были спрятаны на острове.": 'Флинт',
    "Кто увел шхуну у пиратов и спрятал ее в бухте?": 'Джим Хокинс',
    "Кто выкопал сокровища и спрятал их у себя?": 'Бен Ганн',
    "Кого автор представлял как сторонников беззакония и смуты?": 'Джона Сильвера и пиратов',
}


emb = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")
# emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(docs, emb)

k = 2
vector_retriever = vector_store.as_retriever(search_kwargs={"k": k})

bm25_retriever = BM25Retriever.from_documents(docs, k=k)

# инициализация гибридного retriever+reranker
retriver = HybridRerankerRetriever(
    first_retriever=vector_store.as_retriever(search_kwargs={"k": 5}),
    second_retriever=BM25Retriever.from_documents(docs, k=5),
    reranker=SimpleReranker("BAAI/bge-reranker-base"),
    k=k
    )

retrivers = {'Vector': vector_retriever, 'BM25': bm25_retriever, 'hybrid':retriver}

prompt = ChatPromptTemplate.from_template("Ответь на вопрос, без воды, используя только следующий контекст:\nКонтекст: {context}\nВопрос: {question}\nОтвет:")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [6]:
# проверяем
for name_retr, retr in retrivers.items():
    print(f"=== {name_retr} ===")
    # Precision@k
    precision_total = 0
    for question, true_docs in ground_truth_docs.items():
        found_docs = retr.invoke(question)
        found_ids = [d.metadata["source"] for d in found_docs[:k]]

        # Считаем, сколько релевантных нашли
        relevant_found = sum(1 for doc_id in true_docs if doc_id in found_ids)
        precision = relevant_found / k
        precision_total += precision


    precision_avg = precision_total / len(ground_truth_docs)
    print(f"\tСредняя Precision@{k}: {precision_avg:.2f}")

    # Recall@k
    recall_total = 0

    for question, true_docs in ground_truth_docs.items():
        found_docs = retr.invoke(question)
        found_ids = [d.metadata["source"] for d in found_docs[:k]]

        relevant_found = sum(1 for doc_id in true_docs if doc_id in found_ids)
        recall = relevant_found / len(true_docs) if true_docs else 0

        recall_total += recall

    recall_avg = recall_total / len(ground_truth_docs)
    print(f"\tСредняя Recall@{k}: {recall_avg:.2f}")

    chain = (
            {"context": retr | format_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
    )

    llm_answer = []
    for q, true_a in predicted_answers.items():
        response = chain.invoke(q)
        llm_answer.append(response.content)

    scores_llm = []
    for query, reference, prediction in zip(predicted_answers.keys(), predicted_answers.values(), llm_answer):
        score, verdict = evaluate_qa_with_llm(
            query=query,
            prediction=prediction,
            reference=reference
        )
        scores_llm.append(score)

    avg_score_llm = sum(scores_llm) / len(scores_llm)
    print(f"\tСредняя оценка (LLM): {avg_score_llm:.2f}")

=== Vector ===
	Средняя Precision@2: 0.35
	Средняя Recall@2: 0.50
	Средняя оценка (LLM): 0.60
=== BM25 ===
	Средняя Precision@2: 0.25
	Средняя Recall@2: 0.40
	Средняя оценка (LLM): 0.60
=== hybrid ===
	Средняя Precision@2: 0.30
	Средняя Recall@2: 0.45
	Средняя оценка (LLM): 0.50


In [ ]:
#'''
# гибрид просто усреднил векторное и классическое ранжирование'''. Особо точности не привнесло. Когда один из ретриверов плохой.